In [ ]:
# ============================= FORM ============================= #
# @markdown <h2>📂 **Mount Google Drive Configuration**</h2>
# @markdown <h3>📝 Do you want to mount Google Drive?</h3>
mount_google_drive = True  # @param {type:"boolean"}
# @markdown <h4>💼 Rclone Configuration Name:</h4>
rclone_config_name = "gdrive"  # @param {type:"string"}
# @markdown <h4>📂 Local Mount Location:</h4>
local_mount_location = "/content/gdrive/"  # @param {type:"string"}
# @markdown <h4>📂 Drive Mount Location:</h4>
drive_mount_location = "/plexcloud"  # @param {type:"string"}

# @markdown <hr>

# @markdown <h2>📥 **Download Configuration**</h2>
# @markdown <h4>📝 File Name with Extension:</h4>
file_name = "Extra Ordinary Man (2023) 1080P.mkv"  # @param {type:"string"}
# @markdown <h4>📝 File URL or Magnet Link:</h4>
url = "magnet:?xt=urn:btih:5665ef87b09858677a7072751691d52d86d45006&dn=www.5MovieRulz.pet%20-%20Extra%20Ordinary%20Man%20(2023)%201080p%20Telugu%20HQ%20WEB-DL%20-%20AVC%20-%20(DD5%2b5.1%20-%20192kbps%20%26%20AAC)%20-%203.8GB%20Esub.mkv&tr=udp%3a%2f%2ftracker.openbittorrent.com%3a80%2fannounce&tr=udp%3a%2f%2ftracker.opentrackr.org%3a1337%2fannounce&tr=udp%3a%2f%2fp4p.arenabg.com%3a1337%2fannounce&tr=udp%3a%2f%2ftracker.torrent.eu.org%3a451%2fannounce&tr=udp%3a%2f%2ftracker.dler.org%3a6969%2fannounce&tr=udp%3a%2f%2fopen.stealth.si%3a80%2fannounce&tr=udp%3a%2f%2fopentracker.i2p.rocks%3a6969%2fannounce&tr=http%3a%2f%2ftracker.gbitt.info%3a80%2fannounce&tr=udp%3a%2f%2ftracker.tiny-vps.com%3a6969%2fannounce&tr=udp%3a%2f%2fmovies.zsw.ca%3a6969%2fannounce" #@param {type:"string"}
# @markdown <h4>📂 Destination Path:</h4>
download_path = "/content/gdrive/"  # @param {type:"string", description:"Path where the file will be downloaded."}

# @markdown <hr>

# @markdown <h2>⚙️ **Other Configurations**</h2>
# @markdown <h4>📝 True to stop disconnecting:</h4>
stop_disconnect = True  # @param {type:"boolean"}
# @markdown <h4>📝 GitHub raw URL of rclone.conf:</h4>
rclone_url = "https://raw.githubusercontent.com/satyatulasijalandharch/colab/main/rclone.conf"  # @param {type:"string"}

# @markdown <hr>

# @markdown <h2>🚀 **Download Options**</h2>
# @markdown <h4>📝 Direct Download and Rename:</h4>
direct_download = False  # @param {type:"boolean"}
# @markdown <h4>📝 Torrent Download and Rename:</h4>
torrent_download = True # @param {type:"boolean"}
# ============================= FORM ============================= #

# Import necessary libraries
import os
import urllib.parse
import shutil
import time
import IPython
from IPython.display import HTML, display, clear_output

# Install required packages without displaying the output
!python -m pip install --upgrade pip setuptools wheel

# Function to download and set up rclone.conf
def download_and_setup_rclone_conf(rclone_url):
    try:
        # Download rclone.conf from the provided GitHub raw URL
        !curl -sSLo /root/.rclone.conf {rclone_url}

        # Set appropriate permissions
        !chmod 666 /root/.rclone.conf

        # Set the default timezone to Asia/Kolkata
        !sudo ln -sf /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
        !sudo dpkg-reconfigure -f noninteractive tzdata
        clear_output(wait=True)

        # Check if rclone is installed
        if os.path.isfile("/usr/bin/rclone"):
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#00b24c;\">Config has been changed.</h2><br></center>"))
        else:
            !rm -rf /content/sample_data/
            !curl -s https://rclone.org/install.sh | sudo bash
            clear_output(wait=True)
            return display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#446785;\">Installation has been successfully completed.</h2><br></center>"))
    except Exception as e:
        clear_output(wait=True)
        return display(HTML(f"<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;\">Error occurred during setup: {str(e)}</h2><br></center>"))

# Check rclone installation
download_and_setup_rclone_conf(rclone_url)

# Mount Google Drive if the boolean is True
if mount_google_drive:
    # Install FUSE3 without displaying the output
    !sudo apt-get install -y fuse3
    
    # Remove nohup.out if it exists
    if os.path.exists("nohup.out"):
        os.remove("nohup.out")

    # Unmount first
    !fusermount -u $local_mount_location

    # Mount without waiting for the command to complete
    !mkdir $local_mount_location
    !nohup rclone mount $rclone_config_name:$drive_mount_location $local_mount_location --buffer-size 96M &

    # Add a delay to wait for Google Drive to mount
    time.sleep(10)  # Adjust the delay time as needed

    # Show the output that was written to nohup
    time.sleep(3)

    # Check if nohup.out exists before attempting to open it
    if os.path.exists("nohup.out"):
        f = open(r"nohup.out", "r")
        nohupText = f.read()
        f.close()
        dirs = os.listdir(local_mount_location)
        
        if len(dirs) > 0:
            clear_output(wait=True)
            print("Succeeded. ", str(len(dirs)), "dirs found at", local_mount_location)
        else:
            print("\n\nNot succeeded. No files or directories in the mounted location. \nCheck your config name and content. If the rclone command was not found, run the cell above.\n\n")
            print("Log:\n", nohupText)
    else:
        print("nohup.out does not exist. Google Drive may not be mounted.")

# Local download path before moving to rclone mount
local_download_path = "/content/"

# Create local download path if it doesn't exist
if not os.path.exists(local_download_path):
    os.makedirs(local_download_path)

# Torrent Download and Rename if the boolean is True
if torrent_download:
    !python -m pip install lbry-libtorrent
    clear_output(wait=True)

    import libtorrent as lt
    import time
    import datetime

    ses = lt.session()
    ses.listen_on(6881, 6891)
    params = {
        'save_path': (local_download_path),
        'storage_mode': lt.storage_mode_t(2)}

    print(url)

    handle = lt.add_magnet_uri(ses, url, params)
    ses.start_dht()

    begin = time.time()
    print(datetime.datetime.now())

    print ('Downloading Metadata...')
    while (not handle.has_metadata()):
        time.sleep(1)
    print ('Got Metadata, Starting Torrent Download...')
    time.sleep(10)
    print("Starting", handle.name())

    while (handle.status().state != lt.torrent_status.seeding):
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata', \
                'downloading', 'finished', 'seeding', 'allocating']
        print ('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
                (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
                s.num_peers, state_str[s.state]))
        time.sleep(5)

    end = time.time()
    print(handle.name(), "COMPLETE")

    # Specify the new file name
    new_file_name = file_name

    # Rename the downloaded file without specifying the file extension
    old_file_path = os.path.join(local_download_path, handle.name())
    new_file_path = os.path.join(local_download_path, new_file_name)

    # Rename the file
    os.rename(old_file_path, new_file_path)

    print("File renamed to:", new_file_name)
    print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
    print(datetime.datetime.now())

    # Move the file to the rclone mount location
    rclone_dest_path = os.path.join(download_path, new_file_name)
    shutil.move(new_file_path, rclone_dest_path)

    print("File moved to rclone mount:", rclone_dest_path)

# Display a message if no option is selected
if not (stop_disconnect or mount_google_drive or direct_download or torrent_download):
    display(HTML("<center><h2 style=\"font-family:Trebuchet MS;color:#ce2121;font-weight:bold;\">No option selected. Please choose an option to perform.</h2><br></center>"))
